In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbmlStep registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
import numpy as np 

y = np.array([0.5, 1.0, 0.75, 0.60, 0.66])

average_y = np.mean(y)

# Calculate the MSE for each value in y with respect to the average
mse_scores = (y - average_y) ** 2

mse_scores, average_y

(array([0.040804, 0.088804, 0.002304, 0.010404, 0.001764]), 0.7020000000000001)

In [3]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 

In [4]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'amici']

In [5]:
biomodel_id = 'BIOMD0000000744'
document = {
    'sbml': {
        '_type': 'step',
        'address': 'local:get_sbml',
        'config': {
            'biomodel_id': biomodel_id
        },
        'inputs': {
                'biomodel_id': ['biomodel_id_store']
        },
        'outputs': {
                'sbml_model_fp': ['sbml_model_fp_store'],                             
        }
    },
    'ode_compare': {
        '_type': 'step',
        'address': 'local:compare_ode_step',
        'inputs': {
            'data': ['sbml_model_fp_store']
        },
        'outputs': {
            'comparison': ['comparison_store']
        }
    },
    'simulation': {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': 'comparison_store'
                },
                'method': 'lsoda',
                'species_context': 'concentrations'
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                    'names': 'list[string]'
                },
            },
            'inputs': {
                'comparison': 'comparison_store',
                'names': ['names_store']
            }
        }
    } 
}
    


In [6]:
def generate_composite(doc: dict) -> Composite:
    return Composite(config={'state': doc}, core=CORE)

def get_output(doc: dict, dur: int = 1) -> dict:
    comp = generate_composite(doc)
    comp.run(dur)
    return comp.gather_results()

def generate_outputs(docs: list[dict], dur: int = 1) -> list[dict]:
    outputs = []
    for doc in docs:
        result = get_output(doc, dur)
        outputs.append(result)
    return outputs 
    
def run_workflow(biomodel_id: str, dur: int = 10):
    import tempfile
    dirpath = tempfile.mkdtemp()
    document_a = {
        'sbml': {
            '_type': 'step',
            'address': 'local:get_sbml_step',
            'config': {
                'biomodel_id': biomodel_id,
                'save_dir': dirpath
            },
            'inputs': {
                    'biomodel_id': ['biomodel_id_store']
            },
            'outputs': {
                    'sbml_model_fp': ['sbml_model_fp_store'],                             
            }
        },
        'ode_compare': {
            '_type': 'step',
            'address': 'local:compare_ode_step',
            'inputs': {
                'data': ['sbml_model_fp_store']
            },
            'outputs': {
                'comparison': ['comparison_store']
            }
        },
        'emitter_a': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'comparison': 'string',
                },
            },
            'inputs': {
                'comparison': ['comparison_store'],
            }
        }
        
    }
    # sbml_fp = get_output(document_a, 1)[('emitter_a',)][0]['comparison']
    # print(sbml_fp)
    wf1 = Composite(config={'state': document_a}, core=CORE)
    wf1.run(1)
    sbml_fp = wf1.gather_results()[('emitter_a',)][0]['comparison']
    
    document_b = {
        'copasi': {
            '_type': 'process',
            'address': 'local:copasi',
            'config': {
                'model': {
                    'model_source': sbml_fp
                },
                'method': 'lsoda',
                'species_context': 'concentrations',
                'state': wf1.state
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'model_parameters': ['model_parameters_store'],
                'time': ['time_store'],
                'reactions': ['reactions_store']
            },
            'outputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
                'time': ['time_store']
            }
        },
        'emitter_b': {
            '_type': 'step',
            'address': 'local:ram-emitter',
            'config': {
                'emit': {
                    'floating_species_concentrations': 'tree[float]'
                },
            },
            'inputs': {
                'floating_species_concentrations': ['floating_species_concentrations_store'],
            }
        }
    }
    
    print(sbml_fp)
    wf = Composite(config={'state': document_b}, core=CORE)
    wf.run(dur)
    results = wf.gather_results()
    pp(results)
    return results 
    

In [7]:
results = run_workflow(biomodel_id, duration)

results

/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpoqlmrp2h/BIOMD0000000744.xml
found a filepath
{'biomodel_id_store': '', 'sbml': {'_type': 'step', 'address': 'local:get_sbml_step', 'config': {'biomodel_id': 'BIOMD0000000744', 'save_dir': '/var/folders/yy/8crj8x7x5_3b86f0js6_0bn00000gr/T/tmpoqlmrp2h'}, 'inputs': {'biomodel_id': None}, 'outputs': {'sbml_model_fp': None}, 'instance': None, '_inputs': {'biomodel_id': 'string'}, '_outputs': {'sbml_model_fp': 'string'}}, 'ode_compare': {'_type': 'step', 'address': 'local:compare_ode_step', 'inputs': {'data': None}, 'outputs': {'comparison': None}, 'instance': None, 'config': {}, '_inputs': {'data': 'string'}, '_outputs': {'comparison': 'string'}}, 'emitter_a': {'_type': 'step', 'address': 'local:ram-emitter', 'config': {'emit': {'comparison': 'string'}}, 'inputs': {'comparison': None}, 'instance': None, '_inputs': {'comparison': 'string'}, '_outputs': {}, 'outputs': {}}, 'global_time': None, 'sbml_model_fp_store': '/var/folders/yy/8crj8x7

/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/processes/copasi_process.py:207: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



{('emitter_b',): [{'floating_species_concentrations': {'x': 0.0,
    'y': 0.0,
    'z': 0.0,
    'w': 0.0,
    'v': 0.0}},
  {'floating_species_concentrations': {'x': 4.561206356842886e+29,
    'y': 3.3349051597179873e+27,
    'z': 1.121570997999801e+29,
    'w': 1.0269512321590651e+30,
    'v': 8.120385674906026e+29}},
  {'floating_species_concentrations': {'x': 8.894682689109316e+29,
    'y': 6.64387434913788e+27,
    'z': 2.2209542753524913e+29,
    'w': 2.9872793747975995e+30,
    'v': 2.3928001516460926e+30}},
  {'floating_species_concentrations': {'x': 1.3127260459622673e+30,
    'y': 9.936839371983868e+27,
    'z': 3.2985891780714354e+29,
    'w': 5.819505827654842e+30,
    'v': 4.700805600537062e+30}},
  {'floating_species_concentrations': {'x': 1.7310313081635705e+30,
    'y': 1.3222553101197773e+28,
    'z': 4.354906352550734e+29,
    'w': 9.471915062367523e+30,
    'v': 7.696294047688565e+30}},
  {'floating_species_concentrations': {'x': 2.147180772346399e+30,
    'y': 1.650

In [8]:
wf = Composite(config={'state': document}, core=CORE)

Exception: process "get_sbml" not found

In [ ]:
wf.run(10)
state = wf.gather_results()

In [ ]:
state

In [ ]:
state

In [ ]:
from biosimulator_processes.io import fetch_sbml_file
sbml_file = fetch_sbml_file(biomodel_id)

In [ ]:
sbml_file

In [ ]:
from basico import load_model, get_species

model = load_model(sbml_file)


model

In [ ]:
get_species(model=model)